## Congestion domain

This python notebook shows the basic usage of the unified planning library + social laws extension to describe and solve a simple problem.

[![Open In GitHub](https://img.shields.io/badge/see-Github-579aca?logo=github)](https:///github.com/aiplan4eu/unified-planning/blob/master/docs/notebooks/01-basic-example.ipynb)
[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/aiplan4eu/unified-planning/blob/master/docs/notebooks/01-basic-example.ipynb)

## Setup


### Imports

In [2]:
from up_social_laws.ma_problem_waitfor import MultiAgentProblemWithWaitfor
from unified_planning.shortcuts import *
from unified_planning.model.contingent_problem import *
from unified_planning.model.multi_agent import *
import up_fast_downward
unified_planning.environment.get_environment().credits_stream = None

## Problem representation

In this example, we will model a very simple robot navigation problem.

### Types

The first thing to do is to introduce a "UserType" to model the concept of a location. It is possible to introduce as many types as needed; then, for each type we will define a set of objects of that type.  

In addition to `UserType`s we have three built-in types: `Bool`, `Real` and `Integer`.

In [3]:
problem = MultiAgentProblemWithWaitfor("congestion")

In [4]:
Location = UserType('Location')

### Fluents and constants

In [5]:
problem.ma_environment.clear_fluents()

# Location fluent
robot_at = Fluent('robot_at', BoolType(), l=Location)

# Free location fluent
free = Fluent('free', BoolType(), l=Location)
# free = problem.ma_environment.add_fluent('free',
#                                         BoolType(),
#                                         l=Location,
#                                         default_initial_value=True)
problem.ma_environment.add_fluent(free, default_initial_value=True)

# Adjacency fluent
connected = problem.ma_environment.add_fluent('connected', 
                                              BoolType(), 
                                              l_from=Location, 
                                              l_to=Location, 
                                              default_initial_value=False)

### Actions

In [6]:
# Define the action move(l_from, l_to)
move = InstantaneousAction('move', l_from=Location, l_to=Location)

# Define the parameters and preconditions
l_from = move.parameter('l_from')
l_to = move.parameter('l_to')
move.add_precondition(connected(l_from, l_to))
move.add_precondition(free(l_to))
move.add_precondition(robot_at(l_from))

# Define the effects
move.add_effect(robot_at(l_from), False)
move.add_effect(free(l_from), True)
move.add_effect(free(l_to), False)
move.add_effect(robot_at(l_to), True)

## Creating the problem

### Adding locations

In [7]:
NLOC = 11
locations = [Object(f'l{i+1}', Location) for i in range(NLOC)]
problem.add_objects(locations)

### Location adjacency

Describing the location adjacency list as per [Overleaf document](https://www.overleaf.com/project/6485ea4fc610a58da86e2178).

![picture 0](https://i.imgur.com/zHkWzky.png)  

In [8]:
connections = [
    (1, 2), (2, 3), (3, 4),
    (1, 5), (2, 5), (3, 5), (4, 5),
    (1, 6), (2, 6), (3, 6), (4, 6),
    (5, 7), (6, 7),
    (7, 8), (7, 9), (7, 10), (7, 11),
    (8, 9), (9, 10), (10, 11)
]

for i, j in connections:
    problem.set_initial_value(connected(locations[i-1], locations[j-1]), True)
    print(f'{i} -> {j}')

1 -> 2
2 -> 3
3 -> 4
1 -> 5
2 -> 5
3 -> 5
4 -> 5
1 -> 6
2 -> 6
3 -> 6
4 -> 6
5 -> 7
6 -> 7
7 -> 8
7 -> 9
7 -> 10
7 -> 11
8 -> 9
9 -> 10
10 -> 11


### Agents

Now we're setting the agents. We need robots to be in locations `l1-l4`, and get to locations `l8-l11` in the end.

In [9]:
problem.clear_agents()
problem.clear_goals()

robots_number = 4
robots = []

# for i in range(robots_number):
#     robot = Agent(f'robot{i+1}', problem)
#     robots.append(robot)
#     robot.add_public_fluent(robot_at)
#     robot.add_action(move)
#     problem.add_agent(robot)

# for index, robot in enumerate(robots):
#     problem.set_initial_value(Dot(robot, robot_at(locations[index])), True)
#     problem.set_initial_value(free(locations[index]), False)

for i in range(robots_number):
    robot = Agent(f'robot{i+1}', problem)
    # robots.append(robot)
    problem.add_agent(robot)
    robot.add_public_fluent(robot_at, default_initial_value=False)
    robot.add_action(move)
    problem.set_initial_value(Dot(robot, robot_at(locations[i])), True)
    problem.set_initial_value(free(locations[i]), False)
    # problem.add_goal(Dot(robot, robot_at(locations[i+7])))

### Final checks

Checking that our robots have been planted correctly:

In [ ]:
# for i, v in enumerate(robots):
#     print(Dot(v, robot_at(locations[i])))

And checking that our locations have been connected correctly:

In [10]:
# Prints a list of all the location connections sorted alphabetically
sorted([f'{key}: {value}' for key, value in problem.explicit_initial_values.items() if (key)])

['connected(l1, l2): true',
 'connected(l1, l5): true',
 'connected(l1, l6): true',
 'connected(l10, l11): true',
 'connected(l2, l3): true',
 'connected(l2, l5): true',
 'connected(l2, l6): true',
 'connected(l3, l4): true',
 'connected(l3, l5): true',
 'connected(l3, l6): true',
 'connected(l4, l5): true',
 'connected(l4, l6): true',
 'connected(l5, l7): true',
 'connected(l6, l7): true',
 'connected(l7, l10): true',
 'connected(l7, l11): true',
 'connected(l7, l8): true',
 'connected(l7, l9): true',
 'connected(l8, l9): true',
 'connected(l9, l10): true',
 'free(l1): false',
 'free(l2): false',
 'free(l3): false',
 'free(l4): false',
 'robot1.robot_at(l1): true',
 'robot2.robot_at(l2): true',
 'robot3.robot_at(l3): true',
 'robot4.robot_at(l4): true']

### Goals

Proving that we don't have any methods to add goals for an agent:

In [11]:
# print([method for method 
#        in dir(problem.agents[0]) 
#        if not method.startswith('_') 
#        and 'add' in method])

for method in dir(problem.agents[0]):
    if not method.startswith('_') and 'add' in method:
        print(method)

add_action
add_actions
add_fluent
add_fluents
add_private_fluent
add_private_fluents
add_private_goal
add_public_fluent
add_public_fluents
add_public_goal


Defining the goals for robots:

In [12]:
problem.clear_goals()

for index, robot in enumerate(problem.agents):
    current = problem.agent(robot.name)
    # https://unified-planning.readthedocs.io/en/latest/problem_representation.html#multiagent-example
    # problem.add_goal(Equals(Dot(current, robot_at), locations[index+7]))
    # The only way that seems to work, BUT still getting AttributeError: MA_SINGLE_AGENT_PROJECTION
    # print(current)
    # problem.add_goal(Dot(current, robot_at(locations[index+7])))
    # A hacky way to add public goals, BUT the print(problem doesn't work)
    # https://github.com/aiplan4eu/up-social-laws/blob/master/docs/notebook/Social_laws_usecase.ipynb
    # current.public_goals += robot_at(locations[index+7])
    # example from tests/test-social-law.py
    current.add_public_goal(current.fluent("robot_at")(locations[index+7]))

Checking our goals had been set correctly:

In [13]:
# Overall
print(problem.goals, end='\n\n')
# Per-agent
try:
    for agent in problem.agents:
        print(f"{agent.name} ⇢ {agent.public_goals}")
except AttributeError:
    print(problem)

[]

robot1 ⇢ [robot_at(l8)]
robot2 ⇢ [robot_at(l9)]
robot3 ⇢ [robot_at(l10)]
robot4 ⇢ [robot_at(l11)]


And printing the problem to check everything is as expected:

In [ ]:
problem

## Robustness check (fails)

In [19]:
from up_social_laws.robustness_checker import *

# activate up-fast-downward planner
slrc = SocialLawRobustnessChecker()
print(slrc.supported_kind())
sa_result = slrc.is_single_agent_solvable(problem)
result = slrc.is_robust(problem)

print(sa_result)
print("status=", result.status)
print("counter example (original actions): ", result.counter_example_orig_actions)
print("counter example (compiled actions): ", result.counter_example)

PROBLEM_CLASS: ['ACTION_BASED', 'ACTION_BASED_MULTI_AGENT']
TIME: ['TIMED_EFFECTS', 'CONTINUOUS_TIME', 'TIMED_GOALS', 'INTERMEDIATE_CONDITIONS_AND_EFFECTS', 'DURATION_INEQUALITIES']
EXPRESSION_DURATION: ['STATIC_FLUENTS_IN_DURATIONS']
CONDITIONS_KIND: ['DISJUNCTIVE_CONDITIONS', 'NEGATIVE_CONDITIONS', 'UNIVERSAL_CONDITIONS', 'EQUALITIES', 'EXISTENTIAL_CONDITIONS']
EFFECTS_KIND: ['CONDITIONAL_EFFECTS']
TYPING: ['HIERARCHICAL_TYPING', 'FLAT_TYPING']
FLUENTS_TYPE: ['OBJECT_FLUENTS', 'NUMERIC_FLUENTS']
QUALITY_METRICS: ['ACTIONS_COST']
MULTI_AGENT: ['AGENT_SPECIFIC_PUBLIC_GOAL', 'AGENT_SPECIFIC_PRIVATE_GOAL']
False
status= SocialLawRobustnessStatus.NON_ROBUST_SINGLE_AGENT
counter example (original actions):  None
counter example (compiled actions):  None


## Applying social laws

As per the [proposed solution](https://www.overleaf.com/read/hmvfmxswjrfd).

In [20]:
import up_social_laws
print(up_social_laws.__version__)

from up_social_laws.social_law import SocialLaw

l = SocialLaw()

for agent in problem.agents:
    # Use only strings because this should work on any given problem, and note the extra comment to make ("l2",) a tuple (as ("l2") is interpreted as "l2" )
    l.add_waitfor_annotation(agent.name, "move", "free", ("l_to",))
    
new_problem = l.compile(problem).problem

print(new_problem)

0.1.0
problem name = sl_congestion

types = [Location]

environment fluents = [
  bool free[l=Location]
  bool connected[l_from=Location, l_to=Location]
]

agents = [
  Agent name = robot1

private fluents = [
]

public fluents = [
 bool robot_at[l=Location]
]

actions = [
 action move(Location l_from, Location l_to) {
    preconditions = [
      connected(l_from, l_to)
      free(l_to)
      robot_at(l_from)
    ]
    effects = [
      robot_at(l_from) := false
      free(l_from) := true
      free(l_to) := false
      robot_at(l_to) := true
    ]
  }
]

private goals = [
]

public goals = [
 robot_at(l8)
]


  Agent name = robot2

private fluents = [
]

public fluents = [
 bool robot_at[l=Location]
]

actions = [
 action move(Location l_from, Location l_to) {
    preconditions = [
      connected(l_from, l_to)
      free(l_to)
      robot_at(l_from)
    ]
    effects = [
      robot_at(l_from) := false
      free(l_from) := true
      free(l_to) := false
      robot_at(l_to) := true


### Verifying the new problem

In [21]:
result = slrc.is_robust(new_problem)

print(result.status)
print(result.counter_example)

SocialLawRobustnessStatus.NON_ROBUST_SINGLE_AGENT
None


### Trying to solve

In [23]:


result = slrc.solve(new_problem)
print(result.status)
print(result.plan)

AttributeError: module 'unified_planning.engines.results' has no attribute 'UNSOLVABLE_INCOMPLETELY'

Looks like the problem was solved successfully

> #### TODO: 
> - [ ] Plot the solution
> - [x] Fix the problem with social law generation and robustness check / planners
> - [x] Use fast-downward planner
> - [ ] Fix the problem?